In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='.........................', project_access_token='...........................')
pc = project.project_context


### testing box connection within watson studio and creating pandas dataframe from xls file uploaded in box

1. create a Create a custom app in Box https://www.ibm.com/docs/en/cloud-paks/cp-data/3.5.0?topic=catalogs-box-connection and generate the json file with config settings<br> 
2. no need to establish connection in watson studio, yet to post box app config file as asset in watson studio <br>
3. Once the file is accessible, then a box client connection can be created from setting files <br>
4. Once the connection is done, then .content() instruction is used to download the file <br>
5. Lastly, read_excel() is used to write the pandas dataframe 

In [2]:
import pandas as pd
import numpy as np
import os, sys
from ibm_botocore.client import Config
import ibm_boto3

!pip install "boxsdk[jwt]"
from boxsdk import Client
from boxsdk import JWTAuth


/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 130 kB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 4.9 MB/s  eta 0:00:01


In [3]:
# REFERENCE HERE credentials of the json file created as per point 1. above

credentials = {
    'IAM_SERVICE_ID': '..........',
    'IBM_API_KEY_ID': '..........',
    'ENDPOINT': '.......',
    'IBM_AUTH_ENDPOINT': '.........',
    'BUCKET': '............',
    'FILE': '455328_eru3t25n_config.json'
}


In [4]:
cos = ibm_boto3.client(service_name='s3',
ibm_api_key_id=credentials['IBM_API_KEY_ID'],
ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
config=Config(signature_version='oauth'),
endpoint_url=credentials['ENDPOINT'])

In [5]:
from ibm_botocore.client import Config
import ibm_boto3
import types

def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print("File Downloaded")

In [6]:
!rm -rf /home/wsuser/work/*

In [7]:
cos.download_file(Bucket=credentials['BUCKET'],Key='455328_eru3t25n_config.json',Filename='/home/wsuser/work/455328_eru3t25n_config.json')

In [8]:
!pwd

/home/wsuser/work


In [9]:
!ls /home/wsuser/work

455328_eru3t25n_config.json


In [10]:
config = JWTAuth.from_settings_file('/home/wsuser/work/455328_eru3t25n_config.json')
client = Client(config)


myboxfile = client.file('817144087668').get()
print(myboxfile)
file_content = client.file('817144087668').content()

<Box File - 817144087668 (contracts.xlsx)>


In [11]:
df=pd.read_excel(file_content)
df.shape

(6, 11)

### writing dataframe in excel and then posting it to box

In [12]:


from io import BytesIO

output = BytesIO()
writer=pd.ExcelWriter(output, engine='xlsxwriter')
df.to_excel(writer, index=False, sheet_name='sheet')

workbook = writer.book
worksheet1=writer.sheets['sheet']
header_format = workbook.add_format ({ 'bold' : True, 'bg_color': '2EECE6', 'border':1})
worksheet1.freeze_panes(1,0)

def get_col_widths(dataframe): 
    return [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]

for i, width in enumerate(get_col_widths(df)):
    worksheet1.set_column(i, i, width)

workbook.close()
writer.save()
output.seek(0)
assetname="test.xlsx"


project.save_data(assetname, output, set_project_asset=True, overwrite=True)



{'file_name': 'test.xlsx',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmeagitaly-donotdelete-pr-imp2elsawwkf1y',
 'asset_id': 'f3a3fa46-139d-47f2-a3b0-e6dd392fed13'}

In [13]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.

# REFERENCE HERE credentials of the file created as per previous instruction

credentials_1 = {
    'IAM_SERVICE_ID': '..........',
    'IBM_API_KEY_ID': '..........',
    'ENDPOINT': '.......',
    'IBM_AUTH_ENDPOINT': '.........',
    'BUCKET': '............',
    'FILE': 'test.xlsx'



In [14]:


cos = ibm_boto3.client(service_name='s3',
ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
config=Config(signature_version='oauth'),
endpoint_url=credentials_1['ENDPOINT'])



In [15]:
cos.download_file(Bucket=credentials_1['BUCKET'],Key='test.xlsx',Filename='/home/wsuser/work/test.xlsx')

In [16]:
#FIRST TIME
#folder_id="131084746758"
#new_file = client.folder(folder_id).upload('/home/wsuser/work/test.xlsx')
#print('File "{0}" uploaded to Box with file ID {1}'.format(new_file.name, new_file.id))

#SECONTIME TIME ON
file_id = '831822997599'
file_path = '/home/wsuser/work/test.xlsx'

updated_file = client.file(file_id).update_contents(file_path)
print('File "{0}" has been updated'.format(updated_file.name))

File "test.xlsx" has been updated
